In [31]:
## Run in python 2.7+
# Script for all sensor data

import sys

import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber, client_id, client_secret, access_token, folder_id

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

import re
import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates
import altair            as alt

# Enable altair usage in Jupyter Notebook
alt.renderers.enable('notebook')

from   datetime          import timedelta
from   datetime          import datetime

dt = datetime.now()

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters

register_matplotlib_converters()

from   io                import StringIO

import arable_data_dict  as dd

In [8]:
# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

In [9]:
## Run in python 2.7
# Script for all sensor data

import sys

import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber, client_id, client_secret, access_token, folder_id

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

import re
import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates
import altair            as alt

# Enable altair usage in Jupyter Notebook
alt.renderers.enable('notebook')

from   datetime          import timedelta
from   datetime          import datetime

dt = datetime.now()

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters

register_matplotlib_converters()

from   io                import StringIO

import arable_data_dict  as dd

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df


# sensors = [str(i['name']) for i in a.devices]
# Try this instead of the for loop?
sensors = []
for i in a.devices():
    sensors.append(str(i['name']))

print(sensors)

known_sensors        = ['A000522', 'A000536', 'A000502', 'A000563', 'A000507', 'A000229', 'A000513', 'A000465', 'A000318', 'A000369', 'A000466', 'A000546', 'A000514', 'A000417', 'A000461', 'A000543', 'A001512', 'A000502']
deprecated_sensors   = ['A000502', 'A000514', 'A000417', 'A000543', 'A001512', 'A000461']
known_active_sensors = list(set(known_sensors) - set(deprecated_sensors))
known_active_sensors = sorted(known_active_sensors, key=lambda x: int(re.search(r'\d+$',x).group()))

new_active_sensors = list(set(sensors) - set(known_sensors))

new_active_sensors = sorted(new_active_sensors, key=lambda x: int(re.search(r'\d+$',x).group()))


# FOUND NEW SENSOR WITH ID # A000*** ... ERROR - NO DATA AVAILABLE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... ERROR - SENSOR NOT IN RANGE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... SENSOR ENABLED!

if len(new_active_sensors) != 0:
    for i in new_active_sensors:
        print("FOUND NEW SENSOR: ", str(i),)
        for device in a.devices():
            if str(device['name']) == i:
                # Checks sensors are in the correct location (within a specific range of latitude/longitude coordinates).
                date_created   = ymd_hms_to_dt(str(i['created'])[:19])
                date_last_seen = ymd_hms_to_dt(str(i['last_seen'])[:19])
                try:
                    df = arable_query('all','csv',[device_id],"daily","time",dt,dt_to_ymd_hms(dt - timedelta(days = 1)))
                    latitude  = float(str(df['lat' ].values.astype(float)).strip('[ ]'))
                    longitude = float(str(df['long'].values.astype(float)).strip('[ ]'))
                    print("... DATA RECEIVED")
                    if 34.3 <= latitude <= 35.07 and -120.610 <= longitude <= -119.740:
                        print("... SENSOR ENABLED!")
                        known_active_sensors.append(i)
                    else:
                        print("... ERROR - SENSOR NOT IN RANGE!!")
                        # Potentially add something that notifies user of error?
                        
                except:
                    print('... ERROR - NO DATA AVAILABLE!!')
else:
    print("NO NEW SENSORS DETECTED.")
    
print("Retrieving new data from... ")
for i in range(len(known_active_sensors)//4 + 1):
    print('\t'.join(known_active_sensors[i*4:(i+1)*4]) + '\n')

device_id           = [] # Ex. A00****
device_location     = [] # Ex. Lagoon Ice Plants
device_date_created = [] # Ex. 2019-10-10T00:00:00
device_latitude     = []
device_longitude    = []
device_flags        = []
device_info         = []
# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_location.
# The corresponding Device ID is added to device_id.


for i in a.devices():
    for j in known_active_sensors:
        if i['name'] == j:
            device_location.append(str(i['location']['name']))
            device_id.append(str(i['name']))
            flags=[]
            for k in i['flags']:
                flags.append(str(k.strip('[ ]')))
            device_flags.append(flags)
            device_date_created.append(str(i['created'])[:19])
            #device_latitude.append(float(str(df['lat'].values).strip('[ ]')))
            #device_longitude.append(float(str(df['long'].values).strip('[ ]')))
            temp = {'ID'           : str(i['name']),
                    'Location'     : str(i['location']['name'])[1:],
                    'Date Created' : ymd_hms_to_dt(str(i['created'])[:19]),
                    'Flags'        : str(flags),
                    'n'            : len(device_info) + 1}
            device_info.append(temp)

# Converts each item in device_id to a list.          
device_id              = list(map(lambda el:[el], device_id))
device_date_created_dt = [ymd_hms_to_dt(i) for i in device_date_created]

print(['{:<7}'.format(str(i)) for i in device_id])

device_info = pd.DataFrame(device_info)
device_info.to_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")
print(device_info)

['A000536', 'A000563', 'A000507', 'A000466', 'A000369', 'A000318', 'A000522', 'A000546', 'A000465', 'A000229', 'A000513', 'A000502', 'A001512', 'A000514', 'A000417', 'A000461', 'A000543']
NO NEW SENSORS DETECTED.
Retrieving new data from... 
A000229	A000318	A000369	A000465

A000466	A000507	A000513	A000522

A000536	A000546	A000563

["['A000536']", "['A000563']", "['A000507']", "['A000466']", "['A000369']", "['A000318']", "['A000522']", "['A000546']", "['A000465']", "['A000229']", "['A000513']"]
          Date Created                          Flags       ID  \
0  2019-02-06 00:43:30                ['Orientation']  A000536   
1  2019-03-18 22:04:27                ['Orientation']  A000563   
2  2019-03-18 18:45:32                ['Orientation']  A000507   
3  2019-03-12 22:47:11  ['Orientation', 'Thermistor']  A000466   
4  2019-03-12 23:12:28                ['Orientation']  A000369   
5  2019-02-05 23:45:25  ['Orientation', 'Thermistor']  A000318   
6  2019-03-20 19:32:35                [

In [34]:
device_created = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['Date Created'].values.tolist() 
# Monthly
m_start    = []
m_start_fn = []
m_end      = []
m_end_fn   = []

# Yearly
y_start    = []
y_start_fn = []
y_end      = []
y_end_fn   = []

# All Time
astart     = []
start_fn   = []
end        = []
end_fn     = []

for j in range(len(device_created)):
    ms  = []
    msf = []
    me  = []
    mef = []
    
    ys  = []
    ysf = []
    ye  = []
    yef = []
    
    s   = []
    sf  = []
    e   = []
    ef  = []
    
    def new_month(new_m):
        append_ymd_hms(ms, new_m)
        append_string(me, new_m)
        append_ym(msf, new_m)  
        append_ym(mef, new_m)
        
    def new_year(new_y):
        append_ymd_hms(ys, new_y)
        append_string(ye, new_y)
        append_ym(ysf, new_y)
        append_ym(yef, new_y)
    
    device_created_ymd = (device_created[j])[:10]
    start              = ymd_to_dt(device_created_ymd)
    append_string(ms, start)
    append_string(ys, start)
    append_string(s, start)
    append_ym(msf, start)
    append_ym(ysf, start)
    append_ym(sf, start)
    
    start  = start.replace(day=1)
    
    if start.year == dt.year:
        for i in range(start.month+1, dt.month+1):
            new_month(start.replace(month=i))
    
    elif start.year != dt.year: 
        for i in range(dt.year-start.year):
            for j in range(start.month+1, 13): 
                new_month(start.replace(month=j))
        for i in range(start.year+1, dt.year+1):
            newyr = start.replace(year=i, month=1)
            new_year(newyr)
            for j in range(1, dt.month+1):
                new_month(newyr.replace(month=j))
    
    append_string(me, dt)
    append_string(ye, dt)
    append_string(e, dt)
    append_ym(mef, dt)
    append_ym(yef, dt)
    append_ym(ef, dt)
    
    append_temp_list(ms, m_start)
    append_temp_list(me, m_end)
    append_temp_list(msf, m_start_fn)
    append_temp_list(mef, m_end_fn)
    
    append_temp_list(ys, y_start)
    append_temp_list(ye, y_end)
    append_temp_list(ysf, y_start_fn)
    append_temp_list(yef, y_end_fn)
    
    append_temp_list(s, astart)
    append_temp_list(e, end)
    append_temp_list(sf, start_fn)
    append_temp_list(ef, end_fn)
print(m_start)
print(m_end)


[['2019-03-20 00:00:00', '2019-04-01T00:00:00Z', '2019-05-01T00:00:00Z', '2019-06-01T00:00:00Z', '2019-07-01T00:00:00Z', '2019-08-01T00:00:00Z', '2019-09-01T00:00:00Z', '2019-10-01T00:00:00Z', '2019-11-01T00:00:00Z', '2019-12-01T00:00:00Z', '2020-01-01T00:00:00Z', '2020-02-01T00:00:00Z', '2020-03-01T00:00:00Z'], ['2019-02-05 00:00:00', '2019-03-01T00:00:00Z', '2019-04-01T00:00:00Z', '2019-05-01T00:00:00Z', '2019-06-01T00:00:00Z', '2019-07-01T00:00:00Z', '2019-08-01T00:00:00Z', '2019-09-01T00:00:00Z', '2019-10-01T00:00:00Z', '2019-11-01T00:00:00Z', '2019-12-01T00:00:00Z', '2020-01-01T00:00:00Z', '2020-02-01T00:00:00Z', '2020-03-01T00:00:00Z'], ['2019-03-12 00:00:00', '2019-04-01T00:00:00Z', '2019-05-01T00:00:00Z', '2019-06-01T00:00:00Z', '2019-07-01T00:00:00Z', '2019-08-01T00:00:00Z', '2019-09-01T00:00:00Z', '2019-10-01T00:00:00Z', '2019-11-01T00:00:00Z', '2019-12-01T00:00:00Z', '2020-01-01T00:00:00Z', '2020-02-01T00:00:00Z', '2020-03-01T00:00:00Z'], ['2019-03-19 00:00:00', '2019-04-01T

In [41]:
dirpath = 'C:/Users/ccber-arables/Box/'
today = dt
device_ids = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['ID'].values.tolist()    
print(device_ids) 

def write_csv_file(sta, end, data_type, data_type_fn, folder, cm, device): 
    logfile = 'Log_' + str(dt_to_ymd(today)) + '.txt'
    devicee = "['" + str(device) + "']"
    #col = ['sdi12_version', 'sdi12_vendor_id', 'sdi12_sensor_model', 'sdi12_sensor_version', 'sdi12_sensor_sn']
    fn = str(folder) + str(devicee)[2:-2] + '_' + str(cm) + '_' + str(data_type_fn) + '.csv'
    print(fn)
    try:
        df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = devicee, 
                     measure = str(data_type), 
                     order = "time", 
                     end = end, 
                     start = sta, 
                     limit = 100000000)
        df = StringIO(df)
        df = pd.read_csv(df, sep=',', error_bad_lines=False)
        df = df.dropna(axis='columns', thresh=2)#how='all')
        df = df.drop('location', 1)
        #try:
        #    for i in col:
        #        df = df.drop(i, 1)
        #except:
        #    continue
        df.to_csv(fn, sep = ",")
        print('Successfully wrote ' + fn)
        open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
        #box_file = client.folder(folder_id).upload(str(fn), str(fn))
    except:
        open(logfile, 'a+').write('Failure writing ' + fn + '\n')
        pass  

  
# ---> YEAR
# --------> MONTH
# --------------> DATA TYPE
# ------------------------> DEVICE-ID_MONTH-YEAR_DATA-TYPE.csv
for i in range(len(m_start)):
    dev = device_ids[i]
    mm  = m_start[i]
    mn  = m_end[i]
    for k in range(len(mm)):
        s = mm[k]
        e = mn[k]
        Folder_Name          = ymd_to_dt(str(s)[:10]).strftime("%Y")
        print(Folder_Name)
        Sub_Folder_Name      = ymd_to_dt(str(s)[:10]).strftime("%m")
        Sub_Sub_Folder_Names = ["Daily", "Hourly", "Soil", "Calibrated"]
        Data_Types           = ["daily", "hourly", "aux_raw", "health"]

        Folder_Names         = [str(dirpath + str(Folder_Name) + '/' + str(Sub_Folder_Name) + '/' + str(i) + '/') for i in Sub_Sub_Folder_Names]

        import os
        foldername = os.path.basename(dirpath)
        print(foldername)

        for j in range(len(Data_Types)):
            if not os.path.exists(Folder_Names[j]):
                os.makedirs(Folder_Names[j])
                print "Directory " , Folder_Names[j] ,  " Created "
            else:    
                print "Directory " , Folder_Names[j] ,  " already exists"  
                pass
            print(ymd_to_dt(str(e)[:10]))
            print(dt_to_ym(ymd_to_dt(str(s)[:10])))
            write_csv_file(s, ymd_to_dt(str(e)[:10]), Data_Types[j], Sub_Sub_Folder_Names[j], Folder_Names[j], dt_to_ym(ymd_to_dt(str(s)[:10])), dev)

['A000522', 'A000465', 'A000546', 'A000513', 'A000318', 'A000507', 'A000563', 'A000369', 'A000536', 'A000466', 'A000229']
2019

Directory  C:/Users/ccber-arables/Box/2019/03/Daily/  already exists
2019-04-01 00:00:00
2019-03
C:/Users/ccber-arables/Box/2019/03/Daily/A000522_2019-03_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/03/Daily/A000522_2019-03_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/03/Hourly/  already exists
2019-04-01 00:00:00
2019-03
C:/Users/ccber-arables/Box/2019/03/Hourly/A000522_2019-03_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/03/Hourly/A000522_2019-03_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/03/Soil/  already exists
2019-04-01 00:00:00
2019-03
C:/Users/ccber-arables/Box/2019/03/Soil/A000522_2019-03_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/03/Calibrated/  already exists
2019-04-01 00:00:00
2019-03
C:/Users/ccber-arables/Box/2019/03/Calibrated/A000522_2019-03_Calibrated.csv
2019

Directory  C:/Users/c

Successfully wrote C:/Users/ccber-arables/Box/2019/12/Hourly/A000522_2019-12_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/12/Soil/  Created 
2020-01-01 00:00:00
2019-12
C:/Users/ccber-arables/Box/2019/12/Soil/A000522_2019-12_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/12/Soil/A000522_2019-12_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/12/Calibrated/  Created 
2020-01-01 00:00:00
2019-12
C:/Users/ccber-arables/Box/2019/12/Calibrated/A000522_2019-12_Calibrated.csv
2020

Directory  C:/Users/ccber-arables/Box/2020/01/Daily/  Created 
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Daily/A000522_2020-01_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Daily/A000522_2020-01_Daily.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Hourly/  Created 
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Hourly/A000522_2020-01_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Hourly/A000522_2020-01_Hourly.csv
D

2019

Directory  C:/Users/ccber-arables/Box/2019/08/Daily/  already exists
2019-09-01 00:00:00
2019-08
C:/Users/ccber-arables/Box/2019/08/Daily/A000465_2019-08_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/08/Daily/A000465_2019-08_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/08/Hourly/  already exists
2019-09-01 00:00:00
2019-08
C:/Users/ccber-arables/Box/2019/08/Hourly/A000465_2019-08_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/08/Hourly/A000465_2019-08_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/08/Soil/  already exists
2019-09-01 00:00:00
2019-08
C:/Users/ccber-arables/Box/2019/08/Soil/A000465_2019-08_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/08/Calibrated/  already exists
2019-09-01 00:00:00
2019-08
C:/Users/ccber-arables/Box/2019/08/Calibrated/A000465_2019-08_Calibrated.csv
2019

Directory  C:/Users/ccber-arables/Box/2019/09/Daily/  already exists
2019-10-01 00:00:00
2019-09
C:/Users/ccber-arables/Box/2019/09/Daily/A0004

2019

Directory  C:/Users/ccber-arables/Box/2019/04/Daily/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Daily/A000546_2019-04_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Daily/A000546_2019-04_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Hourly/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Hourly/A000546_2019-04_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Hourly/A000546_2019-04_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Soil/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Soil/A000546_2019-04_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Calibrated/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Calibrated/A000546_2019-04_Calibrated.csv
2019

Directory  C:/Users/ccber-arables/Box/2019/05/Daily/  already exists
2019-06-01 00:00:00
2019-05
C:/Users/ccber-arables/Box/2019/05/Daily/A0005

Successfully wrote C:/Users/ccber-arables/Box/2020/01/Daily/A000546_2020-01_Daily.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Hourly/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Hourly/A000546_2020-01_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Hourly/A000546_2020-01_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Soil/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Soil/A000546_2020-01_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Soil/A000546_2020-01_Soil.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Calibrated/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Calibrated/A000546_2020-01_Calibrated.csv
2020

Directory  C:/Users/ccber-arables/Box/2020/02/Daily/  already exists
2020-03-01 00:00:00
2020-02
C:/Users/ccber-arables/Box/2020/02/Daily/A000546_2020-02_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/02/Daily/A0005

...

Successfully wrote C:/Users/ccber-arables/Box/2019/03/Soil/A000536_2019-03_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/03/Calibrated/  already exists
2019-04-01 00:00:00
2019-03
C:/Users/ccber-arables/Box/2019/03/Calibrated/A000536_2019-03_Calibrated.csv
2019

Directory  C:/Users/ccber-arables/Box/2019/04/Daily/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Daily/A000536_2019-04_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Daily/A000536_2019-04_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Hourly/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Hourly/A000536_2019-04_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Hourly/A000536_2019-04_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Soil/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Soil/A000536_2019-04_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Soil/A00053

Successfully wrote C:/Users/ccber-arables/Box/2019/12/Soil/A000536_2019-12_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/12/Calibrated/  already exists
2020-01-01 00:00:00
2019-12
C:/Users/ccber-arables/Box/2019/12/Calibrated/A000536_2019-12_Calibrated.csv
2020

Directory  C:/Users/ccber-arables/Box/2020/01/Daily/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Daily/A000536_2020-01_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Daily/A000536_2020-01_Daily.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Hourly/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Hourly/A000536_2020-01_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Hourly/A000536_2020-01_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Soil/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Soil/A000536_2020-01_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2020/01/Soil/A00053

2019

Directory  C:/Users/ccber-arables/Box/2019/09/Daily/  already exists
2019-10-01 00:00:00
2019-09
C:/Users/ccber-arables/Box/2019/09/Daily/A000466_2019-09_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/09/Daily/A000466_2019-09_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/09/Hourly/  already exists
2019-10-01 00:00:00
2019-09
C:/Users/ccber-arables/Box/2019/09/Hourly/A000466_2019-09_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/09/Hourly/A000466_2019-09_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/09/Soil/  already exists
2019-10-01 00:00:00
2019-09
C:/Users/ccber-arables/Box/2019/09/Soil/A000466_2019-09_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/09/Soil/A000466_2019-09_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/09/Calibrated/  already exists
2019-10-01 00:00:00
2019-09
C:/Users/ccber-arables/Box/2019/09/Calibrated/A000466_2019-09_Calibrated.csv
2019

Directory  C:/Users/ccber-arables/Box/2019/10/Daily/  alrea

2019

Directory  C:/Users/ccber-arables/Box/2019/04/Daily/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Daily/A000229_2019-04_Daily.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Daily/A000229_2019-04_Daily.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Hourly/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Hourly/A000229_2019-04_Hourly.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Hourly/A000229_2019-04_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Soil/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Soil/A000229_2019-04_Soil.csv
Successfully wrote C:/Users/ccber-arables/Box/2019/04/Soil/A000229_2019-04_Soil.csv
Directory  C:/Users/ccber-arables/Box/2019/04/Calibrated/  already exists
2019-05-01 00:00:00
2019-04
C:/Users/ccber-arables/Box/2019/04/Calibrated/A000229_2019-04_Calibrated.csv
2019

Directory  C:/Users/ccber-arables/Box/2019/05/Daily/  alrea

Directory  C:/Users/ccber-arables/Box/2020/01/Hourly/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Hourly/A000229_2020-01_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Soil/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Soil/A000229_2020-01_Soil.csv
Directory  C:/Users/ccber-arables/Box/2020/01/Calibrated/  already exists
2020-02-01 00:00:00
2020-01
C:/Users/ccber-arables/Box/2020/01/Calibrated/A000229_2020-01_Calibrated.csv
2020

Directory  C:/Users/ccber-arables/Box/2020/02/Daily/  already exists
2020-03-01 00:00:00
2020-02
C:/Users/ccber-arables/Box/2020/02/Daily/A000229_2020-02_Daily.csv
Directory  C:/Users/ccber-arables/Box/2020/02/Hourly/  already exists
2020-03-01 00:00:00
2020-02
C:/Users/ccber-arables/Box/2020/02/Hourly/A000229_2020-02_Hourly.csv
Directory  C:/Users/ccber-arables/Box/2020/02/Soil/  already exists
2020-03-01 00:00:00
2020-02
C:/Users/ccber-arables/Box/2020/02/Soil/A000229_2020-02_Soil

In [ ]:
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
import time
import os
def write_csv_file(sta, end, i, sta_f, end_f, device): 
    #col = ['sdi12_version', 'sdi12_vendor_id', 'sdi12_sensor_model', 'sdi12_sensor_version', 'sdi12_sensor_sn']
    print(str(device)[2:-2] + ' has been active since ' + str(sta[0]) + '.')
    for j in range(len(sta)):
        fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
        try:
            df = a.query(select = 'all', 
                         format = 'csv', 
                         devices = device, 
                         measure = str(i), 
                         order = "time", 
                         end = end[j], 
                         start = sta[j], 
                         limit = 100000000)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            df = df.dropna(axis='columns', thresh=2)#how='all')
            df = df.drop('location', 1)
            #try:
            #    for i in col:
            #        df = df.drop(i, 1)
            #except:
            #    continue
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            box_file = client.folder(folder_id).upload(str(fn), str(fn))
        except:
            print('Error writing ' + fn)
            continue  
        

In [ ]:
write_csv_file(sta, end, i, sta_f, end_f, device)

In [ ]:
# Writes data to a .csv file
# Python 3:
# f_input = raw_input('Write data to a .csv file? (y/n) ')

# Python 2:
f_input = raw_input('Write data to a .csv file? (y/n) ')
if f_input == 'y':
    print('\n Select data type: ')
    print(str('{:<2d}'.format(1) + '{:<8}'.format('Daily')      ))
    print(str('{:<2d}'.format(2) + '{:<8}'.format('Hourly')     ))
    print(str('{:<2d}'.format(3) + '{:<8}'.format('Soil')       ))
    print(str('{:<2d}'.format(4) + '{:<8}'.format('Calibrated') ))
    # d_input = int(raw_input('[1-4]  '))
    d_input = int(input('[1-4]  '))
    
    print('\n Select timeframe:  ')
    print(str('{:<2d}'.format(1) + '{:<20}'.format('Year')))
    print(str('{:<2d}'.format(2) + '{:<20}'.format('Month')))
    print(str('{:<2d}'.format(3) + '{:<20}'.format('All Time')))
    print(str('{:<2d}'.format(4) + '{:<20}'.format('Last Week')))
    # t_input = int(raw_input('[1-4]  '))
    t_input = int(input('[1-4]  '))
    
    # all_q = raw_input('\n All devices? (y/n) ')
    all_q = raw_input('\n All devices? (y/n) ')
    if all_q == 'y':
        for i in range(len(device_ids)):
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "health", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "health", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "health", start_fn[i], end_fn[i], device_ids[i])
            if t_input == 4:
                if d_input == 1:
                    write_csv_file(stawk, endwk,      "daily", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 2:
                    write_csv_file(stawk, endwk,     "hourly", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 3:
                    write_csv_file(stawk, endwk,    "aux_raw", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 4:
                    write_csv_file(stawk, endwk, "health", stawk_fn, endwk_fn, device_ids[i])                    
                    
    elif all_q == 'n':
        print('\n Select device: ')
        print('-----------------------------------')
        print(str('{:6}'.format('Number')) + '  ' + '{:<7}'.format('ID') + '  ' + '{:<10}'.format('Location Name'))
        print('-----------------------------------')
        for i in range(len(device_ids)):
            print(str('{:<6d}'.format(i+1)) + '  ' + '{:^7}'.format(str(device_ids[i]).strip("[' ']")) + '  ' + '{:^10}'.format(str(device_names[i]).strip("['Z ']")))
            print('-----------------------------------')
        # j = int(raw_input('[1-' + str(len(device_ids)) + ']  '))
        j = int(input('[1-' + str(len(device_ids)) + ']  '))
        if 1 <= j <= int(len(device_ids)):
            i = j-1
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "health", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "health", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "health", start_fn[i], end_fn[i], device_ids[i])
            if t_input == 4:
                if d_input == 1:
                    write_csv_file(stawk, endwk,      "daily", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 2:
                    write_csv_file(stawk, endwk,     "hourly", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 3:
                    write_csv_file(stawk, endwk,    "aux_raw", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 4:
                    write_csv_file(stawk, endwk, "health", stawk_fn, endwk_fn, device_ids[i])
        else:
            print('User input error')
    
    else:
        print('User input error')
elif f_input == 'n':
    print('\n No files written.')
else:
    print('User input error')
    
items = client.folder(folder_id = folder_id).get_items()
for item in items:
    print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))

In [ ]:
# Make temperature charts for specific timeframe
charts = []
for i in range(len(device_names)):
    de = arable_query('all', 'csv', device_ids[i], "aux_raw", "time", (m_end[i])[-1], (m_start[i])[-1])
    de['time'] = pd.to_datetime(de['time'],infer_datetime_format=True)
    base = alt.Chart(de, 
                     title = str((device_names[i])[2:]) + ' \n ' + str(((m_start[i])[-1])[:10]) + ' -- ' + str((m_end[i])[-1])[:10]
    ).mark_line(color = 'black'
    ).properties(width  = 800, 
                 height = 100
    ).encode(x = alt.X('time:T', axis=alt.Axis(title='')),
             y = alt.Y('analog:Q', axis=alt.Axis(title='Analog')))
    z = alt.Chart(de, 
    ).mark_line(color = 'blue'
    ).properties(width  = 800,
                 height = 80
    ).encode(x = alt.X('time:T', axis=alt.Axis(title='')),
             y = alt.Y('sdi12_value_0:Q', axis=alt.Axis(title='SD112 Value 0')))
    q = alt.Chart(de, 
    ).mark_line(color = 'lightblue'
    ).properties(width  = 800,
                 height = 80
    ).encode(x = alt.X('time:T', axis=alt.Axis(title='')),
             y = alt.Y('sdi12_value_2:Q', axis=alt.Axis(title='SD112 Value 2')))
    graph = alt.vconcat(base, z, q)
    charts.append(graph)
alt.vconcat(*charts)

In [ ]:
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)

fig, ax = plt.subplots(figsize=(3, 3))
ax = plt.gca()

df.plot(kind='line',x='time',y='Tabove',ax=ax)
df.plot(kind='line',x='time',y='Tbelow', color='red', ax=ax)
df.plot(kind='line',x='time',y='Tair', color='green', ax=ax)
df.plot(kind='line',x='time',y='Tdew', color='blue', ax=ax)
plt.xlabel('')
plt.ylabel('Deg. C')
plt.title(str((device_names[0])[1:]) + ' \n ' + str(((sta_m[0])[0])[:10]) + ' -- ' + str((end_m[0])[0])[:10])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()